# Starting and testing OVRO-LWA

### Setup

1. Use dashboard to confirm that services are up for etcd, bifrost pipelines, and data capture.
2. Identify server names for snap2, gpu, and data capture.
3. Set list of server names in notebook.
4. Execute cells


### F-eng
* Repo https://github.com/realtimeradio/caltech-lwa
* python install "control_sw"

### X-eng
* Repo https://github.com/realtimeradio/caltech-bifrost-dsp
* python install "pipeline-control"

### mnc
* Repo https://github.com/ovro-lwa/mnc_python
* python install "mnc_python"

In [7]:
snap2names = ['snap01']  # f-eng
feng_config = '/home/ubuntu/proj/lwa-shell/caltech-lwa/control_sw/config/lwa_corr_config.yaml'  # f-eng and x-eng
xhosts = ['lxdlwagpu02']  # x-eng
npipeline = 4  # x-eng
drips = ['10.41.0.25', '10.41.0.41']  # data recorder (two ips for lxdlwagpu09)
dest_ip = list(sorted(drips*2))  # correlator -> data recorder (destination IPs)
dest_port = [10001+i//npipeline for i in range(npipeline*len(xhosts))]  # correlator -> data recorder (destination ports)
drroot = [f'drvs{ip.split(".")[-1]}' for ip in drips]  # data recorder root names
drids = [f'{drr}{dp-10000:02}' for dp in set(dest_port) for drr in drroot]  # data recorder ids

In [8]:
print(snap2names, xhosts, drips, dest_ip, dest_port, drids)

['snap01'] ['lxdlwagpu02'] ['10.41.0.25', '10.41.0.41'] ['10.41.0.25', '10.41.0.25', '10.41.0.41', '10.41.0.41'] [10001, 10001, 10001, 10001] ['drvs2501', 'drvs4101']


In [9]:
from lwa_f import snap2_fengine
from lwa352_pipeline_control import Lwa352CorrelatorControl
from mnc import mcs, common, ezdr

In [10]:
for snap2name in snap2names:
    lwa_f = snap2_fengine.Snap2Fengine(snap2name)
    lwa_f.cold_start_from_config(feng_config)

2022-03-30 18:38:37,781 - lwa_f.blocks.block:snap01 - WARNING -  - hostname (10.40.0.157) couldn't be turned into integer serial
2022-03-30 18:38:38,495 - lwa_f.blocks.block:snap01 - ERROR - adc - Failed to check ADC control register on fmc 0
2022-03-30 18:38:39,210 - lwa_f.blocks.block:snap01 - ERROR - adc - Failed to check ADC control register on fmc 1
2022-03-30 18:38:39,211 - lwa_f.blocks.block:snap01 - WARNING - eth - Couldn't find Ethernet core. Will retry later
2022-03-30 18:38:39,213 - lwa_f.blocks.block:snap01 - WARNING - powermon - Couldn't initialize because I2C interface not found. Will try again later
2022-03-30 18:38:39,214 - lwa_f.snap2_fengine:snap01 - INFO - Trying to configure output with config file /home/ubuntu/proj/lwa-shell/caltech-lwa/control_sw/config/lwa_corr_config.yaml
2022-03-30 18:38:39,232 - lwa_f.snap2_fengine:snap01 - INFO - Loading existing firmware to snap01
File in flash is:  gpio_pins3_2021-02-07_2206.fpg
2022-03-30 18:38:58,656 - lwa_f.snap2_fengine

RuntimeError: Failed to read size 1 from register sync_ctrl at offset 0

In [5]:
lwa_corr = Lwa352CorrelatorControl(xhosts, npipeline_per_host=npipeline)
lwa_corr.stop_pipelines()   # maybe not needed?
lwa_corr.start_pipelines() 
lwa_corr.configure_corr(dest_ip=dest_ip, dest_port=dest_port)

2021-09-29 23:11:26 [WARNING ] Timeout waiting for pipelines to come up after 60.0 seconds
2021-09-29 23:11:26 [INFO    ] Setting max data output rate per pipeline to 5000.0 Mbit/s
2021-09-29 23:11:26 [INFO    ] Setting pipeline lxdlwagpu02:0 data destination to 10.41.0.25:10001
2021-09-29 23:11:46 [INFO    ] Setting pipeline lxdlwagpu02:1 data destination to 10.41.0.25:10001
2021-09-29 23:11:56 [INFO    ] Setting pipeline lxdlwagpu02:2 data destination to 10.41.0.41:10001
2021-09-29 23:11:56 [INFO    ] Setting pipeline lxdlwagpu02:3 data destination to 10.41.0.41:10001
2021-09-29 23:11:56 [INFO    ] Arming correlator core
2021-09-29 23:11:57 [INFO    ] All pipelines in 'waiting' state as expected
2021-09-29 23:11:57 [INFO    ] Waiting 10 seconds for trigger
2021-09-29 23:12:07 [INFO    ] All pipelines in 'running' state as expected
2021-09-29 23:12:07 [INFO    ] Arming correlator accumulator core
2021-09-29 23:12:08 [INFO    ] All pipelines in 'waiting' state as expected
2021-09-29 23

In [6]:
rs = ezdr.Lwa352RecorderControl('slow')
rs.start()

Found 0 slow recorders: 


[]

### Read data and visualize/test

## Stop

In [9]:
rs.stop()
lwa_corr.stop_pipelines()